In [ ]:
from typing import Dict, List, Tuple
import heapq

In [ ]:
class Node:
    def __init__(self, nid: int, x: float = 0.0, y: float = 0.0, terrain='plano'):
        self.id = nid
        self.x = x
        self.y = y
        self.terrain= terrain

class Edge:
    def __init__(self, u: int, v: int, distance: float, terrain: str):
        self.u = u
        self.v = v
        self.distance = distance
        self.terrain = terrain

In [ ]:
class State:
    def __init__(self, pos: int, battery: float, memory: float, terrain: str):
        self.pos = pos
        self.battery = battery
        self.memory = memory
        self.data = 0.0 
        self.route = []
        self.terrain= terrain

In [ ]:
terrain_consumption = {
    'plano': {'energy_consumption': 1.0},
    'rocas': {'energy_consumption': 2.0},
    'arena': {'energy_consumption': 1.5},
    'dunas': {'energy_consumption': 2.5},
}

def rc_id(r: int, c: int, width: int) -> int:
    return r * width + c

def id_rc(nid: int, width: int) -> Tuple[int,int]:
    return divmod(nid, width)

In [ ]:
class Environment:
    def __init__(self, grid, cell_size_m = 1.0):
        self.h = len(grid)
        self.w = len(grid[0]) if self.h>0 else 0
        self.cell_size = cell_size_m
        self.nodes: Dict[int, Node] = {}
        self.adj: Dict[int, List[Tuple[int, Edge]]] = {}
        self.build(grid)
    
    def build(self, grid):
        moves = [(-1,0),(1,0),(0,-1),(0,1)]

        for i in range(self.h):
            for c in range(self.w):
                val = grid[i][c]
                if val is None or val == 'X':  
                    continue
                if isinstance(val, str):
                    terrain = val
                else:
                    terrain = 'plano'
                nid = rc_id(i,c,self.w)
                self.nodes[nid] = Node(nid, i, c, terrain=terrain)
                self.adj[nid] = []

        for nid, node in list(self.nodes.items()):
            r, c = node.r, node.c
            for dr, dc in moves:
                nr, nc = r+dr, c+dc
                if not (0 <= nr < self.h and 0 <= nc < self.w):
                    continue
                val = grid[nr][nc]
                if val is None or val == 'X':
                    continue
                neighbor_id = rc_id(nr, nc, self.w)
                
                dx = dr * self.cell_size
                dy = dc * self.cell_size
                dist = (dx*dx + dy*dy)**0.5
                
                terrain = self.nodes[neighbor_id].terrain
                edge = Edge(nid, neighbor_id, dist, terrain)
                self.adj[nid].append((neighbor_id, edge))
    

In [ ]:
def cost(edge: Edge, consumption_map=terrain_consumption):
    consumption_terrain = consumption_map.get(edge.terrain, consumption_map['plano'])
    energy = consumption_terrain['energy_consumption'] * edge.distance
    return energy

def dijkstra(world: Environment, start_rc: Tuple[int,int], goal_rc: Tuple[int,int]):
    start = rc_id(start_rc[0], start_rc[1], world.w)
    goal = rc_id(goal_rc[0], goal_rc[1], world.w)
    if start not in world.nodes or goal not in world.nodes:
        return None, float('inf')

    dist = {nid: float('inf') for nid in world.nodes}
    prev = {}
    dist[start] = 0.0
    pq = [(0.0, start)]

    while pq:
        cur_cost, u = heapq.heappop(pq)
        if cur_cost > dist[u]:
            continue
        if u == goal:
            break
        for v, edge in world.adj.get(u, []):
            energy = cost(edge)
            alt = cur_cost + energy
            if alt < dist[v]:
                dist[v] = alt
                prev[v] = u
                heapq.heappush(pq, (alt, v))

    if dist[goal] == float('inf'):
        return None, float('inf')

    path_ids = []
    node = goal
    while node != start:
        path_ids.append(node)
        node = prev[node]
    path_ids.append(start)
    path_ids.reverse()
    path_rc = [id_rc(nid, world.w) for nid in path_ids]
    return path_rc, dist[goal]
